In [ ]:
#installing libraries
%pip install geopy
%pip install xgboost

   ---------------------------------------- 0.0/149.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/149.9 MB ? eta -:--:--
   ---------------------------------------- 1.0/149.9 MB 3.6 MB/s eta 0:00:42
    --------------------------------------- 2.4/149.9 MB 5.0 MB/s eta 0:00:30
   - -------------------------------------- 3.9/149.9 MB 5.6 MB/s eta 0:00:27
   - -------------------------------------- 5.2/149.9 MB 5.9 MB/s eta 0:00:25
   - -------------------------------------- 6.3/149.9 MB 5.7 MB/s eta 0:00:26
   - -------------------------------------- 7.1/149.9 MB 5.5 MB/s eta 0:00:27
   -- ------------------------------------- 7.9/149.9 MB 5.0 MB/s eta 0:00:29
   -- ------------------------------------- 8.7/149.9 MB 4.9 MB/s eta 0:00:29
   -- ------------------------------------- 9.4/149.9 MB 4.9 MB/s eta 0:00:29
   -- ------------------------------------- 10.5/149.9 MB 4.7 MB/s eta 0:00:30
   --- ------------------------------------ 11.3/149.9 MB 4.7 MB/s eta 0:00:3

In [7]:
#data analytics libraries
import numpy as np
import pandas as pd

#data visualizations libraries
import matplotlib.pyplot as plt
import seaborn as sns
#for statistical analysis
import scipy.stats as stats
import statistics
#for finding distance between latitude and longitude
from geopy.distance import geodesic
#for training and testing data
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
#for encoding categorical variables
from sklearn.preprocessing import LabelEncoder,StandardScaler 
#for training models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
#for evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

# Load and Understand Data

In [ ]:
df_train=pd.read_csv("train.csv")
df_train.head(